In [1]:
from pathlib import Path
import os

In [2]:
os.chdir(Path(os.getcwd()).resolve().parents[0])

In [5]:
from src.components import get_ollama_llm, get_mongo_db_client, get_chroma_db_client

In [7]:
mongo_db_client = get_mongo_db_client()
vec_db_client = get_chroma_db_client()
from src.embedding_client import RemoteEmbedding
embed_model = RemoteEmbedding(f"http://localhost:8020")

In [9]:
query = "two men"

In [10]:
embeddings = embed_model.get_query_embedding(query)

In [13]:
len(embeddings[0])

768

In [18]:
chroma_collection = vec_db_client.get_collection('images')

In [20]:
chroma_collection.query(embeddings[0])

{'ids': [['22664',
   '24718',
   '6929',
   '11299',
   '12763',
   '22850',
   '4628',
   '24743',
   '3541',
   '12018']],
 'distances': [[1.4335427,
   1.4344275,
   1.4378197,
   1.4403863,
   1.4437007,
   1.446805,
   1.4492996,
   1.450551,
   1.4509563,
   1.4510105]],
 'embeddings': None,
 'metadatas': [[None, None, None, None, None, None, None, None, None, None]],
 'documents': [['4778593226.jpg',
   '4944016457.jpg',
   '258434606.jpg',
   '3157847991.jpg',
   '3330333217.jpg',
   '4797124142.jpg',
   '2299558545.jpg',
   '4945543678.jpg',
   '2131732576.jpg',
   '3239891140.jpg']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [22]:
from llama_index.vector_stores.chroma import ChromaVectorStore

from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import VectorStoreIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.vector_stores.types import VectorStoreQuery
from llama_index.core.schema import QueryBundle

In [23]:
vec_store = ChromaVectorStore(chroma_collection = chroma_collection)

In [ ]:
from llama_index.core.vector_stores import VectorStoreQuery

In [31]:
vec_store_query = VectorStoreQuery(query_embedding = embeddings[0], similarity_top_k = 5)
results = vec_store.query(query = vec_store_query)

In [35]:
results.nodes

[TextNode(id_='22664', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='4778593226.jpg', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='24718', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='4944016457.jpg', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='6929', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='258434606.jpg', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperato

In [36]:
dir(vec_store)

['__abstractmethods__',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__class_vars__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__fields__',
 '__fields_set__',
 '__format__',
 '__ge__',
 '__get_pydantic_core_schema__',
 '__get_pydantic_json_schema__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__pretty__',
 '__private_attributes__',
 '__pydantic_complete__',
 '__pydantic_computed_fields__',
 '__pydantic_core_schema__',
 '__pydantic_custom_init__',
 '__pydantic_decorators__',
 '__pydantic_extra__',
 '__pydantic_fields__',
 '__pydantic_fields_set__',
 '__pydantic_generic_metadata__',
 '__pydantic_init_subclass__',
 '__pydantic_parent_namespace__',
 '__pydantic_post_init__',
 '__pydantic_private__',
 '__pydantic_root_model__',
 '__pydantic_serializer__',
 '__pydantic_setattr_handl

In [ ]:
index = VectorStoreIndex.from_vector_store(vec_store, embed_model=embed_model)